In [1]:
from glob import glob

import os
import pandas as pd
import numpy as np
import json
from pathlib import Path
import torch
import pickle

In [2]:
order = [
    "DeepFool","FGSM","AutoAttack"
]

In [3]:
dataname = 'CIFAR10'
modelname = 'resnet18'

# Adversarial Attack

In [4]:
del_keys = ['clean acc','noise acc','clean dwt acc']

attack_results = glob(f'./results/{dataname}/saved_adv_samples/{modelname}/*/results.json')
adv_df = pd.DataFrame()

for p in attack_results:
    r = json.load(open(p,'r'))
    
    for k in del_keys:
        del r[k]
    
    r = pd.Series(
        r,
        name = os.path.basename(os.path.dirname(p))
    )
    adv_df = pd.concat([adv_df, r], axis=1)
    
adv_df = ((adv_df.T.loc[order]) * 100).round(2)

adv_df.columns = ['Adv Acc(%)','Adv Acc(%) DWT']

## Successed Images

In [5]:
succ_list = glob(f'./results/{dataname}/saved_adv_samples/{modelname}/*/successed_images.pkl')

In [6]:
for p in succ_list:
    with open(p,'rb') as f:
        nb_imgs = pickle.load(f)['clean'].shape[0]
        adv_method = os.path.basename(os.path.dirname(p))
        adv_df.loc[adv_method,'# Success Images'] = nb_imgs

adv_df['# Success Images'] = adv_df['# Success Images'].astype(int)

In [8]:
print(adv_df.to_markdown())

|            |   Adv Acc(%) |   Adv Acc(%) DWT |   # Success Images |
|:-----------|-------------:|-----------------:|-------------------:|
| DeepFool   |         2.33 |            86.74 |               6871 |
| FGSM       |        10.74 |            14.59 |               6286 |
| AutoAttack |         0    |            55.04 |               7132 |


# Detection

## Known Attack

In [9]:
del_keys = ['TNR','AUIN','AUOUT']

known_results = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/result.json')
known_df = pd.DataFrame()

for p in known_results:
    r = json.load(open(p,'r'))['test']

    for k in del_keys:
        del r[k]

    r = pd.Series(
        r,
        name = os.path.basename(os.path.dirname(p))
    )
    
    known_df = pd.concat([known_df, r], axis=1)

known_df = ((known_df.T.loc[order]) * 100).round(2)

known_df.columns = ['AUROC(%)','Detection Acc(%)']

## Train and Test images

In [10]:
train_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/train.pt')
dev_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/dev.pt')
test_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/test.pt')

In [11]:
for train_p, dev_p, test_p in zip(train_det, dev_det, test_det):
    nb_train = torch.load(train_p)['logits'].size(0)
    nb_dev = torch.load(dev_p)['logits'].size(0)    
    nb_test = torch.load(test_p)['logits'].size(0)
    
    adv_method = os.path.basename(os.path.dirname(train_p))
    
    known_df.loc[adv_method, '#(train, dev, test)'] = f"({nb_train}, {nb_dev}, {nb_test})"

In [12]:
print(known_df.to_markdown())

|            |   AUROC(%) |   Detection Acc(%) | #(train, dev, test)   |
|:-----------|-----------:|-------------------:|:----------------------|
| DeepFool   |      87.76 |              82.54 | (4289, 713, 2148)     |
| FGSM       |      99.99 |              99.54 | (5272, 878, 2639)     |
| AutoAttack |      99.93 |              98.85 | (5598, 932, 2802)     |


## Transfer Attack

In [13]:
transfer_df = pd.read_csv(f'./results/{dataname}/transfer_attack_results/{modelname}/transfer_results.csv',index_col=0)
print(transfer_df.loc[order, order].to_markdown())

|            |   DeepFool |   FGSM |   AutoAttack |
|:-----------|-----------:|-------:|-------------:|
| DeepFool   |      87.76 |  63.8  |        57.34 |
| FGSM       |      56.2  |  99.99 |        93.94 |
| AutoAttack |      22.67 |  22.32 |        99.93 |
